In [0]:
#ONE TIME LOADER FROM HERE BUT THIS IS ONLY SQL QUERY TO TEST WITH MULTIPLE ARTIFACT IDS, HENCE THE S3 KEY NOT NULL IS NOT SET HERE
sqlContext.sql("use mesa_prod.mesa_lake_prod")
df = spark.sql(f'''
 SELECT image_jupiter.id,
    image_artifact_jupiter.kind,
    image_artifact_jupiter.web_s3_bucket,
    image_artifact_jupiter.web_s3_key,
    image_artifact_jupiter.id AS artifact_id
FROM image_jupiter
JOIN image_artifact_jupiter
    ON image_jupiter.id = image_artifact_jupiter.image
ORDER BY group_id
LIMIT 10
'''
)
df.show()

In [0]:
#ONE TIME LOADER FROM HERE
sqlContext.sql("use mesa_prod.mesa_lake_prod")
df = spark.sql(f'''
 SELECT image_jupiter.id,
    image_artifact_jupiter.kind,
    image_artifact_jupiter.web_s3_bucket,
    image_artifact_jupiter.web_s3_key,
    image_artifact_jupiter.id AS artifact_id
FROM image_jupiter
JOIN image_artifact_jupiter
    ON image_jupiter.id = image_artifact_jupiter.image
    WHERE image_artifact_jupiter.web_s3_key IS NOT NULL
ORDER BY group_id
LIMIT 200
'''
)
df.show()

+--------------------+----+-------------+--------------------+--------------------+
|                  id|kind|web_s3_bucket|          web_s3_key|         artifact_id|
+--------------------+----+-------------+--------------------+--------------------+
|61c3ba816b0c478c8...| rgb|    mesa-data| dev/images/helm.png|61c3ba816b0c478c8...|
|61c3ba82909d1e70f...| rgb|    mesa-data| dev/images/helm.png|61c3ba82909d1e70f...|
|61c3ba7f1d52d7187...| rgb|    mesa-data| dev/images/helm.png|61c3ba7f1d52d7187...|
|61c3ba7ed8a3e9349...| rgb|    mesa-data| dev/images/helm.png|61c3ba7ed8a3e9349...|
|61c3ba7dec447bca2...| rgb|    mesa-data| dev/images/helm.png|61c3ba7dec447bca2...|
|61c3ba7ca11c7feb8...| rgb|    mesa-data| dev/images/helm.png|61c3ba7ca11c7feb8...|
|61c3ba81909d1e70f...| rgb|    mesa-data| dev/images/helm.png|61c3ba81909d1e70f...|
|61c3ba7be5100b0f8...| rgb|    mesa-data| dev/images/helm.png|61c3ba7be5100b0f8...|
|61c3ba8291ed7a708...| rgb|    mesa-data| dev/images/helm.png|61c3ba8291ed7a

In [0]:
#us-west-1 	s3.aletheia.imports.prod 608 055
#us-west-2	tartarus.images 60 402 105
#us-west-2	mesa-data 344 794 572

In [0]:
#Assign region to each bucket
from pyspark.sql.functions import when
df2 = df.withColumn("region", when(df.web_s3_bucket == "mesa-data","us-west-2")
                                 .when(df.web_s3_bucket == "tartarus.images","us-west-2")
                                 .when(df.web_s3_bucket == "s3.aletheia.imports.prod" ,"us-west-1"))
df2.show()

+--------------------+----+-------------+--------------------+--------------------+---------+
|                  id|kind|web_s3_bucket|          web_s3_key|         artifact_id|   region|
+--------------------+----+-------------+--------------------+--------------------+---------+
|61c3ba839becf2554...| rgb|    mesa-data| dev/images/helm.png|61c3ba839becf2554...|us-west-2|
|61c3ba7ca11c7feb8...| rgb|    mesa-data| dev/images/helm.png|61c3ba7ca11c7feb8...|us-west-2|
|61c3ba7f1d52d7187...| rgb|    mesa-data| dev/images/helm.png|61c3ba7f1d52d7187...|us-west-2|
|61c3ba7dec447bca2...| rgb|    mesa-data| dev/images/helm.png|61c3ba7dec447bca2...|us-west-2|
|61c3ba82909d1e70f...| rgb|    mesa-data| dev/images/helm.png|61c3ba82909d1e70f...|us-west-2|
|61c3ba7ed8a3e9349...| rgb|    mesa-data| dev/images/helm.png|61c3ba7ed8a3e9349...|us-west-2|
|61c3ba7be5100b0f8...| rgb|    mesa-data| dev/images/helm.png|61c3ba7be5100b0f8...|us-west-2|
|632255f8180be9cb0...| rgb|    mesa-data|prod/images/03dd8..

In [0]:
#Convert Pyspark df to pandas df
import pandas as pd
p_df = df2.toPandas()

In [0]:
#viusalize pandas df, not needed in production though
p_df

,id,kind,web_s3_bucket,web_s3_key,artifact_id,region
0,61c3ba7e6b0c478c85d91374,rgb,mesa-data,dev/images/helm.png,61c3ba7e6b0c478c85d91375,us-west-2
1,61c3ba7be5100b0f858a397c,rgb,mesa-data,dev/images/helm.png,61c3ba7be5100b0f858a397d,us-west-2
2,61c3ba7d8bbdee5f30081319,rgb,mesa-data,dev/images/helm.png,61c3ba7d8bbdee5f3008131a,us-west-2
3,61c3ba816b0c478c85d9138b,rgb,mesa-data,dev/images/helm.png,61c3ba816b0c478c85d9138c,us-west-2
4,61c3ba7f8bbdee5f3008132d,rgb,mesa-data,dev/images/helm.png,61c3ba7f8bbdee5f3008132e,us-west-2
...,...,...,...,...,...,...
195,635393c32c0568c3da9702a5,debayeredrgb,mesa-data,prod/images/7bd7dee5-8cb9-4ce9-af5f-45b1fedccc...,635393c32c0568c3da9702a7,us-west-2
196,635393c286fcaabce957a227,debayeredrgb,mesa-data,prod/images/3f219b13-3654-49ff-8b2c-ac500b3129...,635393c286fcaabce957a229,us-west-2
197,635393c42621341def42f600,debayeredrgb,mesa-data,prod/images/0a2d8a8e-23a4-4b85-9323-9461b28566...,635393c42621341def42f601,us-west-2
198,635393c31b5f06e2f5ad7e9e,debayeredrgb,mesa-data,prod/images/0b136863-8fce-4f49-93d8-e6952e103a...,635393c31b5f06e2f5ad7e9f,us-west-2


In [0]:
#check length
len(p_df)

Out[6]: 200

In [0]:
#get all unique ids from pandas column, not needed in production
unique = p_df['id'].nunique()
unique

Out[7]: 200

In [0]:
#convert ids to a list
image_ids = p_df['id'].values.tolist()


In [0]:
json_array = []
for id in image_ids:
    a = p_df.loc[p_df['id'] == id]
    artifacts = []
    for index,row in a.iterrows():
        artifacts.append({"id": row['artifact_id'], "kind": row['kind'], "s3_bucket": row['web_s3_bucket'], "s3_key": row['web_s3_key'], "region": row['region'] })
    json_format = {"image_id": id, "artifacts": artifacts }
    json_array.append(json_format)

In [0]:
json_array

Out[10]: [{'image_id': '61c3ba7e6b0c478c85d91374',
  'artifacts': [{'id': '61c3ba7e6b0c478c85d91375',
    'kind': 'rgb',
    's3_bucket': 'mesa-data',
    's3_key': 'dev/images/helm.png',
    'region': 'us-west-2'}]},
 {'image_id': '61c3ba7be5100b0f858a397c',
  'artifacts': [{'id': '61c3ba7be5100b0f858a397d',
    'kind': 'rgb',
    's3_bucket': 'mesa-data',
    's3_key': 'dev/images/helm.png',
    'region': 'us-west-2'}]},
 {'image_id': '61c3ba7d8bbdee5f30081319',
  'artifacts': [{'id': '61c3ba7d8bbdee5f3008131a',
    'kind': 'rgb',
    's3_bucket': 'mesa-data',
    's3_key': 'dev/images/helm.png',
    'region': 'us-west-2'}]},
 {'image_id': '61c3ba816b0c478c85d9138b',
  'artifacts': [{'id': '61c3ba816b0c478c85d9138c',
    'kind': 'rgb',
    's3_bucket': 'mesa-data',
    's3_key': 'dev/images/helm.png',
    'region': 'us-west-2'}]},
 {'image_id': '61c3ba7f8bbdee5f3008132d',
  'artifacts': [{'id': '61c3ba7f8bbdee5f3008132e',
    'kind': 'rgb',
    's3_bucket': 'mesa-data',
    's3_key':